## word2vec 개선
- 계산 자원 문제 : embedding 계층(입력층 계산 낭비 감소 효과), negative sampling 손실 함수(은닉층 이후 계산 낭비 감소 효과)

In [1]:
# embedding 계층 구현

import numpy as np
W = np.arange(21).reshape(7,3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [2]:
W[2]

array([6, 7, 8])

In [3]:
idx = np.array([1,0,3,0]) #원하는 행 번호 입력
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [4]:
# embedding 계층의 forward() 메서드 구현

class Embedding:
    def __init__(self,W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

In [5]:
# backward() 메서드 구현

def backward(self, dout):
    dW, = self.grads
    dW[...] = 0 #배열의 모든 요소를 0으로 초기화

    for i, word_id in enumerate(self.idx):
        dW[self.idx] += dout #중복된 인덱스는 기울기를 더함으로써 누적되도록 = 여러번 등장했다는 의미
    return None

In [6]:
#dot 연산(내적) - 정답에 해당하는 열벡터와 은닉층 뉴런의 내적

class EmbeddingDot:
    def __init__(self,W):
        self.embed = Embedding(W)
        self.params = self.embed.parmas
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh



In [7]:
#negative sampling(단어 빈도수를 기준으로 부정적 예를 샘플링, 긍/부정 모두 손실 구하여 합한 값 = 최종 손실)
## 즉, 선택과 집중('모두'가 아닌 '일부'를 처리 = 효율적인 계산 = 다중분류를 이진분류처럼 취급⭐)

# 확률 분포 샘플링

import numpy as np
np.random.choice(10) #무작위 샘플링

0

In [8]:
words = ['you', 'say', 'goodbye','I','hello','.']
np.random.choice(words)

'.'

In [9]:
print('중복 있음 : ', np.random.choice(words, size=5))
print('중복 없음 : ', np.random.choice(words, size=5, replace=False))

중복 있음 :  ['.' 'goodbye' 'you' 'say' '.']
중복 없음 :  ['I' 'goodbye' 'say' '.' 'hello']


In [10]:
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
np.random.choice(words, p=p)

'you'

In [11]:
# 출현 확률이 낮은 단어를 버리지 않도록(0.75 제곱을 통해 확률이 낮은 단어의 확률을 상승시킬 수 있음)
p = [0.7,0.29,0.01]
new_p = np.power(p,0.75)
new_p /=np.sum(new_p)
print(new_p) #기존 p의 0.01에 비해 결과값에서 0.02로 더 상승함

[0.64196878 0.33150408 0.02652714]


In [12]:
#UnigramSampler 클래스

from ch04.negative_sampling_layer import UnigramSampler

corpus = np.array([0,1,2,3,4,1,2,3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1,3,0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[2 4]
 [4 1]
 [2 4]]


In [13]:
# negative sampling 구현

class negativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5) : 
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

In [14]:
# forward 구현

def forward(self, h, target):
    batch_size = target.shape[0]
    negative_sample = self.sampler.get_negative_sample(target)

    # 긍정적 예 순전파
    score = self.embed_dot_layers[0].forward(h, target)
    correct_label = np.ones(batch_size, dtype=np.int32)
    loss = self.loss_layers[0].forward(score, correct_label) #긍정

    # 부정적 예 순전파
    negative_label = np.zeros(batch_size, dtype=np.int32)
    for i in range(self.sample_size):
        negative_target = negative_sample[:,i]
        score = self.embed_dot_layers[1+i].forward(h, negative_target)
        loss += self.loss_layers[1+i].forward(score, negative_label) #1이상부터 부정
    return loss

In [15]:
# backward 구현

def backward(self, dout=1):
    dh = 0 #입력 벡터에 대한 기울기를 저장하기 위한 변수 초기화
    for l0,l1 in zip(self.loss_layers, self.embed_dot_layers):
        dscore = l0.backward(dout) #dout : 역전파 시에 해당 층으로부터 전달되는 미분값(기울기)
        dh += l1.backward(dscore)
    return dh

In [16]:
# 신경망 모델 적용(CBOW)

import sys
sys.path.append('..')
import numpy as np
from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        #가중치 초기화
        W_in = 0.01 * np.random.randn(V,H).astype('f')
        W_out = 0.01 * np.random.randn(V,H).astype('f')

        #계층 생성
        self.in_layers = []
        for i in range(2*window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        #모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [],[]
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        #인스턴스 변수에 단어의 분산 표현을 저장
        self.word_vecs=W_in
        

In [17]:
# forward, backward

def forward(self, contexts, target):
    h=0
    for i, layer in enumerate(self.in_layers):
        h += layer.forward(contexts[:,i])
    h *= 1/len(self.in_layers) #각 레이어의 출력을 더한 후 평균을 계산(정규화) = 입력 레이어 각각의 출력에 대한 평균적인 특징을 고려하여 전체적인 학습을 진행
    loss = self.ns_loss.forward(h, target)
    return loss

def backward(self, dout=1):
    dout = self.ns_loss.backward(dout)
    dout *= 1/len(self.in_layers)
    for layer in self.in_layers:
        layer.backward(dout)
    return None

In [ ]:
# CBOW 모델 학습 코드

import sys
sys.path.append('..')
import numpy as np
from common import config
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from ch04.cbow import CBOW
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb

# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vesc'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f: 
    pickle.dump(params, f, -1) #dump : 파이썬 객체를 직렬화하여 파일에 저장 / -1: 가장 최신의 프로토콜 버전을 사용


In [24]:
# CBOW 모델 평가
import sys
sys.path.append('..')
from common.util import most_similar
import pickle

pkl_file = './ch04/cbow_params.pkl' #상위 코드는 cpu 로 오랜 시간이 걸리므로 pickle 파일로 확인

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

queries = ['you', 'year', 'car','toyota']

for query in queries:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)


[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625


In [25]:
# 벡터 연산으로 유추 문제 풀이

from common.util import analogy

analogy('king','man','queen', word_to_id, id_to_word, word_vecs)
analogy('take','took','go', word_to_id, id_to_word, word_vecs)
analogy('car','cars','child', word_to_id, id_to_word, word_vecs)
analogy('good','better','bad', word_to_id, id_to_word, word_vecs)


[analogy] king:man = queen:?
 woman: 5.16015625
 veto: 4.9296875
 ounce: 4.69140625
 earthquake: 4.6328125
 successor: 4.609375

[analogy] take:took = go:?
 went: 4.55078125
 points: 4.25
 began: 4.09375
 comes: 3.98046875
 oct.: 3.90625

[analogy] car:cars = child:?
 children: 5.21875
 average: 4.7265625
 yield: 4.20703125
 cattle: 4.1875
 priced: 4.1796875

[analogy] good:better = bad:?
 more: 6.6484375
 less: 6.0625
 rather: 5.21875
 slower: 4.734375
 greater: 4.671875
